In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from pathlib import Path
import pandas as pd
import numpy as np
import math

import seaborn as sns

from downstream.signals.signal_r2_permutation_test import collect_paths
from downstream.signals.signal_pca_fit_error_pvalue_permutation_test import process

/Users/romeo/anaconda/envs/aging3/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [77]:
def pvalue_for(normalization, signals_root, simulations, threads):
    output_path = signals_root / "validate.{}.pvalue.{}.csv".format(normalization, simulations)
    print("Results file:", str(output_path), "[exists]" if output_path.exists() else "[not exists]")

    if not output_path.exists():
        print("  calculate:", str(output_path.name))

        # Collect files:
        paths_filter = "_{}.tsv".format(normalization)
        all_paths = collect_paths(signals_root)
        #datatypes = ["H3K4me1", "H3K4me3", "H3K27ac", "H3K36me3", "H3K27me3", "meth"]
        paths = [p for p in all_paths if paths_filter in p.name]
        print("Selected Paths: ", len(paths), "of", len(all_paths))

        # Calc pvalues:
        process(paths, str(output_path), seed=100, simulations=simulations, threads=threads, fdr=True)
        
    return output_path

In [80]:
def fdr_control(normalization, signals_root, fdr=0.05,  simulations = 10000, threads = 4):
    df_path = pvalue_for(normalization, signals_root, simulations, threads)
    df = pd.DataFrame.from_csv(df_path, index_col=None)
    df["loci"] = [Path(f).name for f in df.file]
    print("First pvalue failed FDR control:")
    print(*list(zip(df.columns, df.iloc[next(i for i, t in enumerate(df.pvalue_corr >= fdr) if t),:])),
         sep="\n")
    
    fdf = df[df.pvalue_corr < fdr]
    print("\nPasses FDR control:", len(fdf))
    return  fdf

# Histone Modifications

In [74]:
signals_root = Path("/mnt/stripe/bio/experiments/signal_experiments")

In [69]:
fdr_control("rawq", signals_root, simulations = 100001, threads = 8)

Results file: /mnt/stripe/bio/experiments/signal_experiments/validate.rawq.pvalue.100001.csv [exists]
First pvalue failed FDR control:
('modification', 'meth')
('file', '/mnt/stripe/bio/experiments/signal_experiments/meth/cuffdiff_g_transcript')
('normalization', 'rawq')
('pvalue', 0.0009699806003879922)
('pvalue_corr', 0.13150165568117209)
('loci', 'cuffdiff_g_transcript')

Passes FDR control: 12


,modification,file,normalization,pvalue,pvalue_corr,loci
0,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,cpg_minavcov10_complex_4outliers.narrow.adjust...
1,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diffReps_H3K27ac_both
2,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diffReps_broad_H3K27ac_both
3,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,conservation_regions_0.7_0.8
4,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diffReps_H3K27me3_both
5,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diffReps_broad_input_H3K27me3_both
6,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diff_OD_YD_H3K27me3_zinbra_200_0.01_10_both
7,H3K4me1,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diff_OD_YD_H3K4me1_zinbra_200_1.0E-4_10_both
8,H3K4me3,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,diff_OD_YD_H3K4me3_zinbra_200_0.05_5_both
9,H3K4me3,/mnt/stripe/bio/experiments/signal_experiments...,rawq,0.00001,0.001582,conservation_regions_0.6_0.7


In [70]:
fdr_control("fripz", signals_root, simulations = 100001, threads = 8)

Results file: /mnt/stripe/bio/experiments/signal_experiments/validate.fripz.pvalue.100001.csv [exists]
First pvalue failed FDR control:
('modification', 'H3K4me1')
('file', '/mnt/stripe/bio/experiments/signal_experiments/H3K4me1/cpg_minavcov10_complex_4outliers.narrow.adjusted.regions.filtered')
('normalization', 'fripz')
('pvalue', 0.00032999340013199736)
('pvalue_corr', 0.065256194876102472)
('loci', 'cpg_minavcov10_complex_4outliers.narrow.adjusted.regions.filtered')

Passes FDR control: 7


,modification,file,normalization,pvalue,pvalue_corr,loci
0,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,cpg_minavcov10_complex_4outliers.narrow.adjust...
1,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diffReps_H3K27ac_both
2,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diffReps_H3K27me3_both
3,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diffReps_broad_input_H3K27me3_both
4,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diff_OD_YD_H3K27me3_zinbra_200_0.01_10_both
5,H3K4me1,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diff_OD_YD_H3K4me1_zinbra_200_1.0E-4_10_both
6,H3K4me3,/mnt/stripe/bio/experiments/signal_experiments...,fripz,0.00001,0.00226,diff_OD_YD_H3K4me3_zinbra_200_0.05_5_both


In [71]:
fdr_control("fripm", signals_root, simulations = 100001, threads = 8)

Results file: /mnt/stripe/bio/experiments/signal_experiments/validate.fripm.pvalue.100001.csv [exists]
First pvalue failed FDR control:
('modification', 'H3K27me3')
('file', '/mnt/stripe/bio/experiments/signal_experiments/H3K27me3/diffReps_H3K27me3_both')
('normalization', 'fripm')
('pvalue', 0.0009699806003879922)
('pvalue_corr', 0.066717796078861025)
('loci', 'diffReps_H3K27me3_both')

Passes FDR control: 22


,modification,file,normalization,pvalue,pvalue_corr,loci
0,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,cpg_minavcov10_complex_4outliers.narrow.adjust...
1,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diff_OD_YD_H3K27ac_zinbra_200_1.0E-4_10_young
2,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diff_OD_YD_H3K27ac_zinbra_200_1.0E-4_10_both
3,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diffReps_H3K27ac_old
4,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diffReps_H3K27ac_both
5,H3K27ac,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diffReps_broad_H3K27ac_both
6,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diffReps_broad_H3K27me3_young
7,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diff_OD_YD_H3K27me3_zinbra_200_0.01_10_young
8,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diff_OD_YD_H3K27me3_zinbra_200_0.01_10_old
9,H3K27me3,/mnt/stripe/bio/experiments/signal_experiments...,fripm,0.00001,0.000833,diffReps_broad_input_H3K27me3_both


# Input

In [75]:
input_signals_root = Path("/mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws")

In [86]:
norm = "rawq"
fdf = fdr_control(norm, input_signals_root, simulations = 10000, threads = 8)
print("Passed FDR: ", len(fdf))
pd.DataFrame.from_csv(str(input_signals_root / "validate.{}.pvalue.10000.csv".format(norm)), index_col=None).head()

Results file: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/validate.rawq.pvalue.10000.csv [exists]
First pvalue failed FDR control:
('modification', nan)
('file', '/mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cgi_cd14_chromhmm18.hg19.14_TssBiv')
('normalization', 'rawq')
('pvalue', 0.0016998300169983002)
('pvalue_corr', 0.50484951504849518)
('loci', 'cgi_cd14_chromhmm18.hg19.14_TssBiv')

Passes FDR control: 0
Passed FDR:  0


,modification,file,normalization,pvalue,pvalue_corr
0,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawq,0.001700,0.504850
1,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawq,0.018498,0.578701
2,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawq,0.021698,0.578701
3,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawq,0.028097,0.578701
4,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawq,0.028097,0.578701


In [88]:
norm = "rawz"
fdf = fdr_control(norm, input_signals_root, simulations = 10000, threads = 8)
print("Passed FDR: ", len(fdf))
pd.DataFrame.from_csv(str(input_signals_root / "validate.{}.pvalue.10000.csv".format(norm)), index_col=None).head()

Results file: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/validate.rawz.pvalue.10000.csv [not exists]
  calculate: validate.rawz.pvalue.10000.csv
Selected Paths:  297 of 1485
--- [1 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cd14v2_chromhmm18.hg19.5_Tx/cd14v2_chromhmm18.hg19.5_Tx_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [2.0, 6.0]; 50% = 4.0, p-value: 0.36166383361663834
--- [2 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cd14v2_chromhmm18.hg19.8_EnhG2/cd14v2_chromhmm18.hg19.8_EnhG2_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.2752724727527247
--- [3 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27ac_macs_broad_ODS_weak_consensus/H3K27ac_macs_broad_ODS_weak_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [2.0, 6.0]; 50% = 4.0, 

[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.3808619138086191
--- [32 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cd14_chromhmm18.hg19.17_ReprPCWk/cd14_chromhmm18.hg19.17_ReprPCWk_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [2.0, 6.0]; 50% = 4.0, p-value: 0.2920707929207079
--- [33 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cd14_chromhmm18.hg19.18_Quies/cd14_chromhmm18.hg19.18_Quies_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.11208879112088792
--- [34 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/cd14_chromhmm18.hg19.1_TssA/cd14_chromhmm18.hg19.1_TssA_rawz.tsv
[ACTUAL]: 6, [SIMUL]: [min, max] = [0, 9], [2%, 98%] = [2.0, 7.0]; 50% = 4.0, p-value: 0.9416058394160584
--- [35 / 297] -----------
Process: /mnt/stripe/bio/experiments/signa

[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.8144185581441856
--- [62 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_zinbra_ODS_weak_consensus/H3K27me3_zinbra_ODS_weak_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.8336166383361664
--- [63 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_zinbra_weak_consensus/H3K27me3_zinbra_weak_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 9], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.824017598240176
--- [64 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_zinbra_YDS_weak_consensus/H3K27me3_zinbra_YDS_weak_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.8424157584241576
--- [65 / 297] -----------
Process: /mnt/stripe/bio

[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.38166183381661833
--- [92 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/noncoding_exons/noncoding_exons_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.06329367063293671
--- [93 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/noncoding_transcript/noncoding_transcript_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.1352864713528647
--- [94 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/noncoding_tes[-2000..2000]/noncoding_tes[-2000..2000]_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.09769023097690231
--- [95 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/hcp_tss_tss[-5000..-25

[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.34406559344065596
--- [122 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/any_all_H3K27me3_1.0E-80_80_peaks/any_all_H3K27me3_1.0E-80_80_peaks_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [1, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.8760123987601239
--- [123 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/any_all_H3K4me3_1.0E-60_10_peaks/any_all_H3K4me3_1.0E-60_10_peaks_rawz.tsv
[ACTUAL]: 1, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.041695830416958304
--- [124 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/any_all_H3K4me1_1.0E-80_10_peaks/any_all_H3K4me1_1.0E-80_10_peaks_rawz.tsv
[ACTUAL]: 4, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.6552344765523448
--- [125 / 297] -----------
Process: /mnt/stri

...

[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.3848615138486151
--- [212 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diffReps_broad_input_H3K36me3_both/diffReps_broad_input_H3K36me3_both_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.3184681531846815
--- [213 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diff_macs_pooled_H3K36me3_both/diff_macs_pooled_H3K36me3_both_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [1, 8], [2%, 98%] = [2.0, 6.0]; 50% = 4.0, p-value: 0.0872912708729127
--- [214 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diffReps_H3K36me3_both/diffReps_H3K36me3_both_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 9], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.3144685531446855
--- [215 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal

[ACTUAL]: 3, [SIMUL]: [min, max] = [1, 5], [2%, 98%] = [3.0, 5.0]; 50% = 5.0, p-value: 0.23447655234476553
--- [242 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diff_macs_pooled_Y_O_H3K4me3_old/diff_macs_pooled_Y_O_H3K4me3_old_rawz.tsv
[ACTUAL]: 4, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.6256374362563744
--- [243 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diff_OD_YD_H3K4me3_zinbra_200_0.05_5_old/diff_OD_YD_H3K4me3_zinbra_200_0.05_5_old_rawz.tsv
[ACTUAL]: 2, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [2.0, 7.0]; 50% = 4.0, p-value: 0.1456854314568543
--- [244 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diffReps_broad_H3K4me3_old/diffReps_broad_H3K4me3_old_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [1, 7], [2%, 98%] = [1.0, 6.0]; 50% = 4.0, p-value: 0.38166183381661833
--- [245 / 297] -----------
Process: /mnt/str

[ACTUAL]: 4, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.6032396760323968
--- [272 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_macs_broad_YDS_median_consensus/H3K27me3_macs_broad_YDS_median_consensus_rawz.tsv
[ACTUAL]: 3, [SIMUL]: [min, max] = [0, 7], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.2896710328967103
--- [273 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_sicer_median_consensus/H3K27me3_sicer_median_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.7984201579842016
--- [274 / 297] -----------
Process: /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K27me3_sicer_ODS_median_consensus/H3K27me3_sicer_ODS_median_consensus_rawz.tsv
[ACTUAL]: 5, [SIMUL]: [min, max] = [0, 8], [2%, 98%] = [1.0, 7.0]; 50% = 4.0, p-value: 0.8104189581041896
--- [275 / 297] -----------
Pro

34. 0.07849215078492151: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/hcp_tss_tss[-5000..-2500]
35. 0.07929207079292071: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/lcp_tss_tes[-2000..2000]
36. 0.0800919908009199: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diffReps_broad_H3K4me1_both
37. 0.08249175082491751: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/conservation_regions_0.0_0.1
38. 0.08489151084891511: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/OD_on_enhancers
39. 0.08489151084891511: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/H3K4me1_zinbra_ODS_median_consensus
40. 0.0872912708729127: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diff_macs_pooled_H3K36me3_young
41. 0.0872912708729127: [rawz] /mnt/stripe/bio/experiments/signal_input/input_unique_tags_bws/diff_macs_pooled_H3K36me3_both
42. 0.08809119088091

,modification,file,normalization,pvalue,pvalue_corr
0,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawz,0.000900,0.267273
1,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawz,0.003300,0.485051
2,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawz,0.004900,0.485051
3,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawz,0.009699,0.514623
4,NaN,/mnt/stripe/bio/experiments/signal_input/input...,rawz,0.009699,0.514623
